# Group by et Aggrégation 

Grouper un set de données suivant une ou plusieurs dimensions et appliquer une réduction est une composante clé de toute analyse. 

Le langage SQL ainsi que les tableaux croisés dynamiques offrent de premières fonctions de Grouping. 

Dans ce chapitre vous apprendrez : 

- La logique du Groupby avec Pandas
- Comment réaliser des réductions avec des fonctions standards ou customisées
- Vos premières exploration de données

## Group By

In [ ]:
import pandas as pd
import numpy as np

La mécanique du group by peut être résumé suivant le schéma ci-dessous : 
    
   - **Split** : un set de données est séparé suivant des clés
   - **Apply** : une fonction est appliquée sur chacun de ces groupes 
   - **Combine** : les résultats sont ensuite regroupés au sein d'un seul objet 

<img src='files/images/groupby.png' width=500>

> <cite>Source: Python for Data Analysis</cite>

Nous allons utiliser dans cet exemple le set de données ***reserve_parlementaire.csv*** (Il s'agit de la liste des réserves parlementaries mises en ligne en 2011). 

Pour en savoir plus sur la réserve parlementaire - http://www.lemonde.fr/les-decodeurs/article/2015/02/12/le-detail-des-80-millions-de-la-reserve-parlementaire-2014-publie_4575418_4355770.html

Importons ce fichier et inspectons le : 

In [ ]:
reserve = pd.read_csv("data/reserve.csv")
reserve.head(3)

Nous souhaitons savoir quel est le Groupe politique ayant reçu le plus de subventions. 

On souhaite donc **grouper** les données suivant la colonne ***Groupe politique du parlementaire***, puis sommer les données de la colonne ***Subvention allouée***. 

Nous utilisons pour cela la méthode **groupby** :

In [ ]:
partis = reserve['Subvention allouée'].groupby(reserve['Groupe politique du parlementaire'])

print(partis)

L'objet ***partis*** renvoyé est une série groupée. Aucun calcul n'a encore été réalisé. 

Si nous souhaitons sommer les résultats, nous pouvons appliquer une fonction. Ici grâce à la méthode **sum**

In [ ]:
partis.sum()

D'autres aggrégations sont possibles : la méthode **count** permet par exemple de renvoyer la taille d'un groupe (ici le nombre de subventions par partis politiques). 

In [ ]:
partis.count()

### A vous : 

   - Quelle a été la plus grande subvention allouée en 2011 ? 
   - Quel est le parlementaire qui a reçu le plus grand nombre de subventions ? 
   - Quel est le parlementaire qui a reçu le plus de subventions en valeur ? 
   - Quel est le département qui a reçu le plus de subventions ? 

In [ ]:
reserve = pd.read_csv("data/reserve.csv")
reserve.head(2)

Nous pouvons appliquer plusieurs clés : **quel est le nombre de subventions par partis politiques et par chambre ?**

In [ ]:
chambres = reserve['Subvention allouée'].groupby([reserve['Groupe politique du parlementaire'],reserve['Nature']]).count()
chambres

Nous créons dans ce cas une série avec deux index (partis et nature). 

Nous pouvons faire 'déplier' cette série avec la méthode **unstack** :

In [ ]:
groupes_chambres = chambres.unstack()
groupes_chambres

Nous pouvons également grouper sans spécifier de colonne. Si les données à grouper se trouvent dans mon DataFrame, je peux grouper de la façon suivante : 

In [ ]:
reserve.groupby('Nature').count()

(Attention aux aggrégations hâtives ;))

### A vous : 

- Quelle est la moyenne des coûts de projets par type d'assemblée ? 
- Combien de projets le Sénat a - t-il financé dans les Yvelines ? 
- Quelle a été la subvention allouée pour à Paris ? 

Pandas nous permet également de passer des dictionnaires ou des séries au sein des groupes. 

Cela est intéressant lorsque nous souhaitons faire un mapping entre les colonnes (ou les lignes). 

Dans notre cas, les acronymes entre l'Assemblée Nationale et le Sénat varient (i.e : SOC et SRC désignent tous deux les groupes du Parti Socialiste). On peut donc écrire la correspondance suivante : 

In [ ]:
mapping = {'CRC':'Partis de Gauche', 
           'CRC-SPG':'Partis de Gauche', 
           'ECO':'Ecologistes', 
           'GDR':'Radicaux', 
           'NC':'Centristes', 
           'NI':'Non Inscrits', 
           'RDSE':'Radicaux',
           'SOC':'Parti Socialiste',
           'SOCV':'Parti Socialiste',
           'SRC':'Parti Socialiste',
           'UC':'Centristes', 
           'UDI':'Centristes', 
           'UMP':'Union Mouvement Populaire'}

Dans ce cas, nous souhaitons grouper selon la correspondance des index (axis =0) du DataFrame **groupes_chambres** avec le dictionnaire **mapping**, soit : 

In [ ]:
groupes_chambres

In [ ]:
groupes_chambres.groupby(mapping, axis=0).sum()

Une autre façon de réaliser facilement des mapping, est d'utiliser la méthode **map** sur une série. Dans notre cas : 

In [ ]:
reserve['parti'] = reserve['Groupe politique du parlementaire'].map(mapping)
reserve.head(2)

## Agrégation

L'aggrégation de données correspond à une transformation d'un tableau de données en une valeur. 

Pandas permet d'accéder à des agrégations génériques (moyennes, somme etc..), mais il est également possible de créer ses propres fonctions d'agrégation. 

Il est ainsi possible d'aggréger ses données en utilisant plusieurs fonctions d'agrégations. Déclarons par exemple les trois fonctions suivantes moyenne, maximum et minimum : 

In [ ]:
functions = ['mean','max','min']

Nous pouvons appeler la méthode **agg** qui appliquera pour chacun des groupes les fonctions appelées. 

Si nous souhaitons par exemple afficher la moyenne, le maximum et le minimum des subventions allouée par parti politique, nous pouvons écrire : 

In [ ]:
reserve.groupby('parti').agg(functions)

Il est également possible de définir ses propres aggrégations grâce à la méthode **apply**

Comme précisé précédemment, la méthode **apply** sépare un set de données suivant les clés spécifier dans la méthode **groupby** puis applique la fonction appelée en paramètre. 

Nous défninissons la fonction **part** qui renvoie la moyenne du ratio subvention allouée / coût du projet : 

In [ ]:
def part(df):
    return np.mean(df['Subvention allouée']/df['Coût du projet'])

La moyenne totale de la part des projets subventionnées peut donc s'écrire : 

In [ ]:
part(reserve)

Il est dès lors possible d'appliqer la fonction **part** avec un groupby pour obtenir la part des subventions par Département : 

In [ ]:
reserve.groupby('Département').apply(part).sort_values(ascending=False)

### A vous : 

- Quel est la part subventionnée des projets par partis politiques ? 
- Quel parlementaire, ayant réalisé plus de 50 subventions, a réalisé le plus de subventions dans sa région ? 
- Quel est le parti politique qui a réalisé le plus de subventions dans sa région ? 